The [Taylor-Green Vortex](https://en.wikipedia.org/wiki/Taylor%E2%80%93Green_vortex) in 2D is defined with initial conditions

$$ 
u_x = -U\sin(x)\cos(y),\ u_y = U\cos(x)\sin(y)
$$

Let's simulate the evolution of that flow. First we include the packages we need.

In [ ]:
# using Pkg
# Pkg.resolve()
# Pkg.instantiate()

using WaterLily

include("../examples/TwoD_plots.jl") # For unix/OSX systems
# include("..\\examples\\TwoD_plots.jl") # For Windows systems
gr(show = :ijulia)

Now write a function to set up a simulation with the given initial conditions. And a second function to evolove the flow in time and plot the vorticity.

In [ ]:
function TGVortex(;p=7,Re=100)
    # Define vortex size, velocity, viscosity
    L = 2^p; U = 1; ν = U*L/Re

    # Taylor-Green-Vortex initial velocity field
    function uλ(i,vx)
        x,y = @. (vx-1.5)*2π/L          # scaled coordinates
        i==1 && return -U*sin(x)*cos(y) # u_x
        i==2 && return  U*cos(x)*sin(y) # u_y
    end

    # Return simulation
    return Simulation((L+2,L+2),zeros(2),L;uλ,ν,U)
end

function runSim_PlotCurl(sim;tend,clim)
    t₀ = WaterLily.sim_time(sim)
    for t ∈ range(t₀,t₀+tend,step=0.25)
        sim_step!(sim,t)
        @inside sim.flow.σ[I] = WaterLily.curl(3,I,sim.flow.u)*sim.L/sim.U
        flood(sim.flow.σ,clims=(-clim,clim),legend=false,border=:none)
    end
end

Running the cell below evolves this initial condition over time. We visualize the vorticity of the flow by taking the curl of the velocity.

In [ ]:
sim = TGVortex(Re=100) # Low Re, Laminar decay
runSim_PlotCurl(sim,tend=5,clim=2)

Using a low Re, this 2D flow decays steadily over time, with the decay rate set by the Reynolds number.

In [ ]:
sim = TGVortex(Re=10^6) # High Re. Turbulent instability
runSim_PlotCurl(sim,tend=20,clim=10)

At high Re, an instability takes over and the size of the vortex doubles. 

[The 3D version of this flow](https://www.youtube.com/watch?v=5sTt0wRNHkM) evolves completely differently; the symmetry is broken and turbulence emgerges and tears up the large slow vortices into small eddies very quickly.